In [ ]:
%run "standardScalerSetup.ipynb"
%run "minMaxScalerSetup.ipynb"
#loads all the preprocessing libraries and prepares the dataframe

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
from sklearn import metrics

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
#### CNN Model ##########
def format_to_cnn(df):
    X = np.expand_dims(df, axis=2)
    return X

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
inputSize = 9
colList = ['HiddenLayers', 'R2Score', 'MAE', 'MSE', 'H5FileName', 'TrainHistory', 'TrainPredictions']

In [ ]:
#Define CNN model
def cnn_model_mae(inputSize=inputSize):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation=custom_activation,
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     input_shape=(inputSize,1)))
    model.add(Conv1D(filters=64, kernel_size=3, 
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     activation=custom_activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation=custom_activation))
    #model.add(BatchNormalization())
    model.add(Dense(1, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation="linear"))

    optmzr=Adam(lr=0.001)
    model.compile(optimizer=optmzr, loss='mae', metrics=['mae'])

    return model
    
def cnn_model_mse(inputSize=inputSize):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation=custom_activation,
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     input_shape=(inputSize,1)))
    model.add(Conv1D(filters=64, kernel_size=3, 
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     activation=custom_activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation=custom_activation))
    #model.add(BatchNormalization())
    model.add(Dense(1, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation="linear"))

    optmzr=Adam(lr=0.001)
    model.compile(optimizer=optmzr, loss='mse', metrics=['mse'])

    return model


In [ ]:
checkpoint_cnn_std_all_mae = ModelCheckpoint("cnn_model_std_all_mae_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_std_all_mae = [checkpoint_cnn_std_all_mae, reduce_lr]

checkpoint_cnn_std_all_mse = ModelCheckpoint("cnn_model_std_all_mse_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_std_all_mse = [checkpoint_cnn_std_all_mse, reduce_lr]

In [ ]:
# Transform the datasets for LSTM processing
cnn_X_test1_summaryL1Z2_std = format_to_cnn(X_test1_summaryL1Z2_std)
cnn_X_test1_summaryL1Z2_minMax = format_to_cnn(X_test1_summaryL1Z2_minMax)
cnn_X_train_base_std = format_to_cnn(X_train_base_std)
cnn_X_train_base_minMax = format_to_cnn(X_train_base_minMax)
cnn_X_train_all_std = format_to_cnn(X_train_all_std)
cnn_X_train_all_minMax = format_to_cnn(X_train_all_minMax)
cnn_X_test_all_std = format_to_cnn(X_test_all_std)
cnn_X_test_all_minMax = format_to_cnn(X_test_all_minMax)

In [ ]:
cnn_regr_std_all_mae = cnn_model_mae()
cnn_regr_std_all_mse = cnn_model_mse()

In [ ]:
cnn_regr_std_all_mae.summary()

In [ ]:
# STD ALL MAE

In [ ]:
%%time
history_CNN_std_all_mae = cnn_regr_std_all_mae.fit(cnn_X_train_all_std, y_train_scaled_std_all,
                            batch_size=30, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_std_all_mae)

In [ ]:
cnn_regr_std_all_mae_new =load_model('cnn_model_std_all_mae_L2.h5')
#Predict
y_pred_CNN_std_all_mae_scaled = cnn_regr_std_all_mae_new.predict(cnn_X_test_all_std)
#reverse scaling to compare to true values
pred = targetStdScalerAll.inverse_transform(y_pred_CNN_std_all_mae_scaled)
#Show results
print(metrics.r2_score(y_test_all_std, pred))
print(metrics.mean_absolute_error(y_test_all_std, pred))
print(metrics.mean_squared_error(y_test_all_std, pred))

In [ ]:
minMaeAWS = metrics.mean_absolute_error(y_test_all_std, pred)
minR2AWS = metrics.r2_score(y_test_all_std, pred)
modelNameAWS = "CNN_STD_ALL_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_std_all_mae.history['loss'])
plt.plot(history_CNN_std_all_mae.history['val_loss'])
plt.title('CNN MAE ALL STD model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
#plt.scatter(range(y_test_scaled_std_all.shape[0]),y_test_scaled_std_all,label="Original Data", alpha=0.6, c='black')
#plt.scatter(range(y_pred_CNN_std_all_mae_scaled.shape[0]),y_pred_CNN_std_all_mae_scaled,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MAE Std All Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# STD ALL MSE

In [ ]:
%%time
history_CNN_std_all_mse = cnn_regr_std_all_mae.fit(cnn_X_train_all_std, y_train_scaled_std_all,
                            batch_size=30, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_std_all_mse)

In [ ]:
cnn_regr_std_all_mse_new =load_model('cnn_model_std_all_mse_L2.h5')
#Predict
y_pred_CNN_std_all_mse_scaled = cnn_regr_std_all_mse_new.predict(cnn_X_test_all_std)
#reverse scaling to compare to true values
pred = targetStdScalerAll.inverse_transform(y_pred_CNN_std_all_mse_scaled)
#Show results
print(metrics.r2_score(y_test_all_std, pred))
print(metrics.mean_absolute_error(y_test_all_std, pred))
print(metrics.mean_squared_error(y_test_all_std, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_std, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_std, pred)
    minR2AWS = metrics.r2_score(y_test_all_std, pred)
    modelNameAWS = "CNN_STD_ALL_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_std_all_mse.history['loss'])
plt.plot(history_CNN_std_all_mse.history['val_loss'])
plt.title('CNN MSE ALL STD model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_std.shape[0]),y_test_all_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
#plt.scatter(range(y_test_scaled_std_all.shape[0]),y_test_scaled_std_all,label="Original Data", alpha=0.6, c='black')
#plt.scatter(range(y_pred_CNN_std_all_mse_scaled.shape[0]),y_pred_CNN_std_all_mse_scaled,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MSE Std All Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# MinMax All MAE

In [ ]:
cnn_regr_minMax_all_mae = cnn_model_mae()
cnn_regr_minMax_all_mse = cnn_model_mse()

In [ ]:
checkpoint_cnn_minMax_all_mae = ModelCheckpoint("cnn_model_minMax_all_mae_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_minMax_all_mae = [checkpoint_cnn_minMax_all_mae, reduce_lr]

checkpoint_cnn_minMax_all_mse = ModelCheckpoint("cnn_model_minMax_all_mse_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_minMax_all_mse = [checkpoint_cnn_minMax_all_mse, reduce_lr]

In [ ]:
%%time
history_CNN_minMax_all_mae = cnn_regr_minMax_all_mae.fit(cnn_X_train_all_minMax, y_train_scaled_minMax_all,
                            batch_size=30, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_minMax_all_mae)

In [ ]:
cnn_regr_minMax_all_mae_new =load_model('cnn_model_minMax_all_mae_L2.h5')
#Predict
y_pred_CNN_minMax_all_mae_scaled = cnn_regr_minMax_all_mae_new.predict(cnn_X_test_all_minMax)
#reverse scaling to compare to true values
pred = targetMinMaxScalerAll.inverse_transform(y_pred_CNN_minMax_all_mae_scaled)
#Show results
print(metrics.r2_score(y_test_all_minMax, pred))
print(metrics.mean_absolute_error(y_test_all_minMax, pred))
print(metrics.mean_squared_error(y_test_all_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_minMax, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_minMax, pred)
    minR2AWS = metrics.r2_score(y_test_all_minMax, pred)
    modelNameAWS = "CNN_MinMax_ALL_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_minMax_all_mae.history['loss'])
plt.plot(history_CNN_minMax_all_mae.history['val_loss'])
plt.title('CNN MAE ALL MinMax model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_minMax.shape[0]),y_test_all_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
#plt.scatter(range(y_test_scaled_minMax_all.shape[0]),y_test_scaled_minMax_all,label="Original Data", alpha=0.6, c='black')
#plt.scatter(range(y_pred_CNN_minMax_all_mae_scaled.shape[0]),y_pred_CNN_minMax_all_mae_scaled,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MAE MinMax All Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
#MinMax All MSE

In [ ]:
%%time
history_CNN_minMax_all_mse = cnn_regr_minMax_all_mse.fit(cnn_X_train_all_minMax, y_train_scaled_minMax_all,
                            batch_size=30, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_minMax_all_mse)

In [ ]:
cnn_regr_minMax_all_mse_new =load_model('cnn_model_minMax_all_mse_L2.h5')
#Predict
y_pred_CNN_minMax_all_mse_scaled = cnn_regr_minMax_all_mse_new.predict(cnn_X_test_all_minMax)
#reverse scaling to compare to true values
pred = targetMinMaxScalerAll.inverse_transform(y_pred_CNN_minMax_all_mse_scaled)
#Show results
print(metrics.r2_score(y_test_all_minMax, pred))
print(metrics.mean_absolute_error(y_test_all_minMax, pred))
print(metrics.mean_squared_error(y_test_all_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test_all_minMax, pred) < minMaeAWS:
    minMaeAWS = metrics.mean_absolute_error(y_test_all_minMax, pred)
    minR2AWS = metrics.r2_score(y_test_all_minMax, pred)
    modelNameAWS = "CNN_MinMax_ALL_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_minMax_all_mse.history['loss'])
plt.plot(history_CNN_minMax_all_mse.history['val_loss'])
plt.title('CNN MSE ALL MinMax model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test_all_minMax.shape[0]),y_test_all_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
#plt.scatter(range(y_test_scaled_minMax_all.shape[0]),y_test_scaled_minMax_all,label="Original Data", alpha=0.6, c='black')
#plt.scatter(range(y_pred_CNN_minMax_all_mse_scaled.shape[0]),y_pred_CNN_minMax_all_mse_scaled,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MSE MinMax All Model for X_test dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# Base STD

In [ ]:
cnn_regr_std_base_mae = cnn_model_mae()
cnn_regr_std_base_mse = cnn_model_mse()

In [ ]:
checkpoint_cnn_std_base_mae = ModelCheckpoint("cnn_model_std_base_mae_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_std_base_mae = [checkpoint_cnn_std_base_mae, reduce_lr]

checkpoint_cnn_std_base_mse = ModelCheckpoint("cnn_model_std_base_mse_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_std_base_mse = [checkpoint_cnn_std_base_mse, reduce_lr]

In [ ]:
# Base STD MAE

In [ ]:
%%time
history_CNN_std_base_mae = cnn_regr_std_base_mae.fit(cnn_X_train_base_std, y_train_scaled_std_base,
                            batch_size=5, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_std_base_mae)

In [ ]:
cnn_regr_std_base_mae_new =load_model('cnn_model_std_base_mae_L2.h5')
#Predict
y_pred_CNN_std_base_mae_scaled = cnn_regr_std_base_mae_new.predict(cnn_X_test1_summaryL1Z2_std)
#reverse scaling to compare to true values
pred = targetStdScalerBase.inverse_transform(y_pred_CNN_std_base_mae_scaled)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_std, pred))

In [ ]:
minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred)
minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_std, pred)
modelNameAWS_base = "CNN_STD_Base_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_std_base_mae.history['loss'])
plt.plot(history_CNN_std_base_mae.history['val_loss'])
plt.title('CNN MAE Base STD model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MAE Std Base Model for X_test_L1Z2 dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# Base STD MSE

In [ ]:
%%time
history_CNN_std_base_mse = cnn_regr_std_base_mse.fit(cnn_X_train_base_std, y_train_scaled_std_base,
                            batch_size=5, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_std_base_mse)

In [ ]:
cnn_regr_std_base_mse_new =load_model('cnn_model_std_base_mse_L2.h5')
#Predict
y_pred_CNN_std_base_mse_scaled = cnn_regr_std_base_mse_new.predict(cnn_X_test1_summaryL1Z2_std)
#reverse scaling to compare to true values
pred = targetStdScalerBase.inverse_transform(y_pred_CNN_std_base_mse_scaled)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_std, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_std, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_std, pred)
    modelNameAWS_base = "CNN_STD_Base_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_std_base_mse.history['loss'])
plt.plot(history_CNN_std_base_mse.history['val_loss'])
plt.title('CNN MSE Base STD model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_std.shape[0]),y_test1_summaryL1Z2_std,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MSE Std Base Model for X_test_L1Z2 dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# Base MinMax

In [ ]:
cnn_regr_minMax_base_mae = cnn_model_mae()
cnn_regr_minMax_base_mse = cnn_model_mse()

In [ ]:
checkpoint_cnn_minMax_base_mae = ModelCheckpoint("cnn_model_minMax_base_mae_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_minMax_base_mae = [checkpoint_cnn_minMax_base_mae, reduce_lr]

checkpoint_cnn_minMax_base_mse = ModelCheckpoint("cnn_model_minMax_base_mse_L2.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
callbacks_list_cnn_minMax_base_mse = [checkpoint_cnn_minMax_base_mse, reduce_lr]

In [ ]:
# Base MinMax MAE

In [ ]:
%%time
history_CNN_minMax_base_mae = cnn_regr_minMax_base_mae.fit(cnn_X_train_base_minMax, y_train_scaled_minMax_base,
                            batch_size=5, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_minMax_base_mae)

In [ ]:
cnn_regr_minMax_base_mae_new =load_model('cnn_model_minMax_base_mae_L2.h5')
#Predict
y_pred_CNN_minMax_base_mae_scaled = cnn_regr_minMax_base_mae_new.predict(cnn_X_test1_summaryL1Z2_minMax)
#reverse scaling to compare to true values
pred = targetMinMaxScalerBase.inverse_transform(y_pred_CNN_minMax_base_mae_scaled)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_minMax, pred)
    modelNameAWS_base = "CNN_MinMax_Base_MAE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_minMax_base_mae.history['loss'])
plt.plot(history_CNN_minMax_base_mae.history['val_loss'])
plt.title('CNN MAE Base MinMax model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_minMax.shape[0]),y_test1_summaryL1Z2_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MAE MinMax Base Model for X_test_L1Z2 dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
# Base MinMax MSE

In [ ]:
%%time
history_CNN_minMax_base_mse = cnn_regr_minMax_base_mse.fit(cnn_X_train_base_minMax, y_train_scaled_minMax_base,
                            batch_size=5, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_minMax_base_mse)

In [ ]:
cnn_regr_minMax_base_mse_new =load_model('cnn_model_minMax_base_mse_L2.h5')
#Predict
y_pred_CNN_minMax_base_mse_scaled = cnn_regr_minMax_base_mse_new.predict(cnn_X_test1_summaryL1Z2_minMax)
#reverse scaling to compare to true values
pred = targetMinMaxScalerBase.inverse_transform(y_pred_CNN_minMax_base_mse_scaled)
#Show results
print(metrics.r2_score(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred))
print(metrics.mean_squared_error(y_test1_summaryL1Z2_minMax, pred))

In [ ]:
if metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred) < minMaeAWS_base:
    minMaeAWS_base = metrics.mean_absolute_error(y_test1_summaryL1Z2_minMax, pred)
    minR2AWS_base = metrics.r2_score(y_test1_summaryL1Z2_minMax, pred)
    modelNameAWS_base = "CNN_MinMax_Base_MSE"

In [ ]:
#Plot train vs validation
plt.figure(figsize=(20,10))
plt.plot(history_CNN_minMax_base_mse.history['loss'])
plt.plot(history_CNN_minMax_base_mse.history['val_loss'])
plt.title('CNN MSE Base MinMax model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot prediction vs original
plt.figure(figsize=(20,10))
plt.scatter(range(y_test1_summaryL1Z2_minMax.shape[0]),y_test1_summaryL1Z2_minMax,label="Original Data", alpha=0.6, c='black')
plt.scatter(range(pred.shape[0]),pred,label="Predicted Data", 
            alpha=0.6, c='red')
plt.ylabel('Total Messages')
plt.xlabel('Records')
plt.title('CNN MSE MinMax Base Model for X_test_L1Z2 dataset prediction vs original')
plt.legend()
plt.show()

In [ ]:
minMaeAWS, minR2AWS, modelNameAWS

In [ ]:
minMaeAWS_base, minR2AWS_base, modelNameAWS_base